In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pycountry



## Get Top Places to Visit

In [2]:
def getSoup(link):
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'lxml')


### Links


GETTING LIST OF COUNTRIES NAMES

In [3]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2.lower()


#### Country Things to do top 15

In [113]:
def getTripAdvisorLink(country):
    soup = getSoup('https://www.google.com/search?q=tripadvisor+best+things+to+do+in+'+str(country))
    ta = 'http://tripadvisor.com/Attractions' 
    i = 0
    length = len(soup.findAll('h3', {'class':"r"}))
    while True:
        url=soup.findAll('h3', {'class':"r"})[i].contents[0].get('href').partition("Attractions")[2].partition(".html")[0]
        i+=1
        adress = ta+url
        if adress !=  ta:
            return adress
            break
        if i == length :
            break

In [117]:
links={}
for country in countries.keys():
    links[country]= getTripAdvisorLink(country)
print(links)

{'Aruba': 'http://tripadvisor.com/Attractions-g147247-Activities-Aruba', 'Afghanistan': 'http://tripadvisor.com/Attractions-g659499-Activities-Afghanistan', 'Angola': 'http://tripadvisor.com/Attractions-g293762-Activities-Angola', 'Anguilla': 'http://tripadvisor.com/Attractions-g147238-Activities-Anguilla', 'Åland Islands': 'http://tripadvisor.com/Attractions-g189898-Activities-Aland_Island_Aland', 'Albania': 'http://tripadvisor.com/Attractions-g294445-Activities-Albania', 'Andorra': 'http://tripadvisor.com/Attractions-g190391-Activities-Andorra', 'United Arab Emirates': 'http://tripadvisor.com/Attractions-g294012-Activities-United_Arab_Emirates', 'Argentina': 'http://tripadvisor.com/Attractions-g294266-Activities-Argentina', 'Armenia': 'http://tripadvisor.com/Attractions-g293931-Activities-Armenia', 'American Samoa': 'http://tripadvisor.com/Attractions-g60665-Activities-American_Samoa', 'Antarctica': 'http://tripadvisor.com/Attractions-g660183-Activities-Antarctic_Peninsula', 'French 

### Attraction Titles 


In [35]:
attraction_titles_all=[]
for i in links_tripadvisor:    
    soup = getSoup(i)
    divs = soup.findAll('div', {'class':'listing_title'}) 
    text_chunks = {div.contents[1] for div in divs}
    attraction_titles_all.append({title.contents[0] for title in text_chunks})
    

IBA KÓD

In [77]:

soup = getSoup("https://www.tripadvisor.com/Attractions-g295113-Activities-Cocos_Keeling_Islands.html")
divs = soup.findAll('div', {'class':'listing_title'}) 
text_chunks = {div.contents[1] for div in divs}
attraction_titles = {title.contents[0] for title in text_chunks}
print(attraction_titles)

set()
